# Accelerated Database Recovery - Fast rollback and log truncation
In this example you will see how fast rollback can now be with Accelerated Database Recovery. You will also see that a long active transaction does not affect the ability to truncate the transaction log.


## Step 1: Setup the database
Create a database and put a bunch of rows in a table. Make the database SIMPLE recovery to make it easier to analyze log truncation.

The database assumes a path for a standard SQL Server Window installation. The database and log are created at 10Gb each to avoid any autogrow.

100,000 rows are created in the table to be able to show the different in log growth and rollback times.

Depending on the speed of your server, creating the database and data could take several minutes.

**Note**: *For Linux installations the default path to use is /var/opt/mssql*

In [1]:
USE master
GO
DROP DATABASE IF EXISTS gocowboys
GO
CREATE DATABASE gocowboys
ON PRIMARY
(NAME = N'gocowboys_primary', FILENAME = 'C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\DATA\gocowboys.mdf', SIZE = 10Gb , MAXSIZE = UNLIMITED, FILEGROWTH = 65536KB)
LOG ON 
(NAME = N'gocowboys_Log', FILENAME = 'C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\DATA\gocowboys_log.ldf', SIZE = 10Gb , MAXSIZE = UNLIMITED , FILEGROWTH = 65536KB)
GO
ALTER DATABASE gocowboys SET RECOVERY SIMPLE
GO
USE gocowboys
GO
DROP TABLE IF EXISTS howboutthemcowboys
GO
CREATE TABLE howboutthemcowboys (playerid int primary key clustered, playername char(7000) not null)
GO
SET NOCOUNT ON
GO
BEGIN TRAN
DECLARE @x int
SET @x = 0
WHILE (@x < 100000)
BEGIN
	INSERT INTO howboutthemcowboys VALUES (@x, 'Jason Whitten returns in 2019')
	SET @x = @x + 1
END
COMMIT TRAN
GO
SET NOCOUNT OFF
GO
USE master
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:18.338

## Step 2: Delete all the rows in the table
Delete all the rows in the table in a user defined transaction

In [2]:
USE master
GO
ALTER DATABASE gocowboys SET ACCELERATED_DATABASE_RECOVERY = OFF
GO
USE gocowboys
GO
BEGIN TRAN
DELETE from howboutthemcowboys
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(100000 rows affected)

Total execution time: 00:00:05.115

## Step 3: Check how much transaction log space is used
Because there is an active transaction the transaction log will have space used that cannot be truncated

In [3]:
SELECT * FROM sys.dm_db_log_space_usage
GO

(1 row affected)

Total execution time: 00:00:00.115

database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
6,10737410048,4223672320,39.33604,1168621568


## Step 4: Does a checkpoint truncate the transaction log?
Compare the used_log_space_in_percent to the value from the previous step. Normally for a database with simple recovery a CHECKPOINT would truncate the log.

In [4]:
CHECKPOINT
GO
SELECT * FROM sys.dm_db_log_space_usage
GO

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.205

database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
6,10737410048,4223750144,39.33677,1000026112


## Step 5: How long does it take to rollback all the deletes
Try to rollback the transation and measure the time it takes

In [5]:
ROLLBACK TRAN
GO

Commands completed successfully.

Total execution time: 00:00:13.889

## Step 6: What is the log space usage after a CHECKPOINT
Because the active transaciton is rolled back, a CHECKPOINT will allow the log to be truncated.

In [6]:
CHECKPOINT
GO
SELECT * FROM sys.dm_db_log_space_usage
GO

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.894

database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
6,10737410048,65884160,0.6135945,81920


## Step 7: Turn on Accelerated Database Recovery

In [7]:
USE master
GO
ALTER DATABASE gocowboys SET ACCELERATED_DATABASE_RECOVERY = ON
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.014

## Step 8: Try to delete all the rows again under a transaction

In [8]:
USE gocowboys
GO
BEGIN TRAN
DELETE from howboutthemcowboys
GO

Commands completed successfully.

(100000 rows affected)

Total execution time: 00:00:01.655

## Step 9: Check log space usage before and after a CHECKPOINT
Notice the space used for the log is low even before the CHECKPOINT. This is because an active transaction does not affect log truncation when using Accelerated Database Recovery.

In [9]:
SELECT * FROM sys.dm_db_log_space_usage
GO
CHECKPOINT
GO
SELECT * FROM sys.dm_db_log_space_usage
GO

(1 row affected)

database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
6,10737410048,1056620544,9.840553,768172032


Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.687

database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
6,10737410048,385695744,3.592074,131072


## Step 10: How fast is a rollback?
See if a rollback is any faster with Acclerated Database Recovery

In [10]:
ROLLBACK TRAN
GO

Commands completed successfully.

Total execution time: 00:00:00.001